<a href="https://colab.research.google.com/github/Verolynz/Multiple-Disease-Prediction/blob/main/multiple_disease_prediction_98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'multiple-disease-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4532617%2F7752217%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240605%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240605T030235Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5bd0ed6e27a2fc5d5ef6184aa2883ff8fe1317d7b3b04c62f93725a88c40ee0d8f10b31e07efa3532c1a1e55c21001def9e8bc80d105987cb699d792bb373ce06d95b1564215479784089e5d451f2901ef8b519e16ae6fcc3220d1104b19b1afdf9333cc4b9fae1eb9610c5540979ca8b598a58120856e007d7e5c4bc0b4a445c2e421f4111ed1389fec6df195baf6adf24d65566b97fe4be1604ddd6617664854df087360f1831c027cdd04077aafa583e33588e7888cd381d9f57e648ea3599cfac77cca885ecaedc1a9184bac554b107dfaf661020989a3fe6989380de59eaa3ae6549bc16a90568f72cd942a4d41b4d635c6093ee27c66a25b11f8abb333'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Basic Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# warnings removal
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
from sklearn.feature_selection import RFE,SelectKBest,SelectFromModel
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Preprocessing

In [ ]:
# defining dataframes
train_df = pd.read_csv('/kaggle/input/multiple-disease-prediction/Blood_samples_dataset_balanced_2(f).csv')
test_df = pd.read_csv('/kaggle/input/multiple-disease-prediction/blood_samples_dataset_test.csv')

In [ ]:
# checking train_df
train_df.head()

In [ ]:
print('train shape:',train_df.shape)
print('test shape: ',test_df.shape)

In [ ]:
train_df.dtypes

In [ ]:
# Are there any missing values?
print('train_df missing values: ',train_df.isna().sum().sum())
print('test_df missing values: ',test_df.isna().sum().sum())

In [ ]:
# Unique variables in the target column of the training set.
print('-------train set-------')
print(f'Unique variables in the target column: {train_df.Disease.unique()}\nthe number of unique variables in the target column: {train_df.Disease.nunique()}\n')
# Unique variables in the target column of the test set.
print('-------test set-------')
print(f'Unique variables in the target column: {test_df.Disease.unique()}\nthe number of unique variables in the target column: {test_df.Disease.nunique()}')


In [ ]:
# Let's combine 2 data frames.
data = pd.concat([train_df,test_df],ignore_index=True)

In [ ]:
# let's check our new data frame.
data.head()

In [ ]:
data.shape # (rows, columns)

In [ ]:
# Let's find out how many of each class there
data.Disease.value_counts()

In [ ]:
# Let's convert our target column to numeric values.
data.Disease = data.Disease.astype('category')
d = dict(enumerate(data['Disease'].cat.categories))
data.Disease = data.Disease.cat.codes.values
data.Disease

In [ ]:
# Make a correlation matrix
plt.figure(figsize=(20, 10))
heatmap = sns.heatmap(data.corr(), vmin=-1, vmax=1, annot=True,cmap='BrBG')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':14}, pad=12)

2: Healty  
1: Diabetes
4: Thalasse  
0: Anemia  
5: Thromboc  
3: Heart Di

In [ ]:
# Let's find how many of each class there are in the bar chart.
counts = data[data.columns[-1]].value_counts()
palette = sns.color_palette()
fig, axes = plt.subplots(figsize=(10,6))
counts.plot(kind="bar", ax=axes, color=palette)
plt.xlabel('Diseases',labelpad=10)
for container in axes.containers:
    axes.bar_label(container)
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=5, ncols=5, figsize=(15, 15))

# Draw a distplot for each column and add it to the subplot
for i, col in enumerate(data.columns):
    sns.distplot(data[col], ax=axs[i//5, i%5])
    axs[i//5, i%5].set_title(col)

# Hide missing subplots
for i in range(len(data.columns), 25):
    axs[i//5, i%5].axis('off')

#  Show charts
plt.tight_layout()
plt.show()

In [ ]:
# Draw a boxplot for each column
index = 0
for i in range(0, len(data.columns) - 1, 4):
    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16, 4))
    for j, ax in enumerate(axes.flat):
        if index < len(data.columns) - 1:
            sns.boxplot(data=data, x=data.columns[-1], y=data.columns[index], ax=ax)
            ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
            index += 1
    plt.tight_layout()
    plt.show()

In [ ]:
data['Hemoglobin'].describe()

In [ ]:
X = data.drop('Disease',axis=1)
y = data['Disease']

In [ ]:
X

In [ ]:
y

# ANN

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=10,stratify= y)

In [ ]:
results = pd.DataFrame([],columns=['Accuracy','train Accuracy'])

In [ ]:
print('train shape:',X_train.shape)
print('test shape: ',X_test.shape)

In [ ]:
# standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# create ann model
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)))
ann.add(tf.keras.layers.Dropout(0.2))
ann.add(tf.keras.layers.Dense(128, activation='relu'))
ann.add(tf.keras.layers.Dropout(0.2))
ann.add(tf.keras.layers.Dense(64,activation='relu'))
ann.add(tf.keras.layers.Dropout(0.2))
ann.add( tf.keras.layers.Dense(6, activation='softmax'))

In [ ]:
# let's compile our model.
ann.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# Let's add an early stop command.
early_stopping = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True, monitor='val_loss')

In [ ]:
# model fit
history = ann.fit(X_train_scaled, y_train, epochs=100,
                    validation_split=0.1, callbacks=[early_stopping], verbose=2)

In [ ]:
print('test set',ann.evaluate(X_test_scaled,y_test)) # test set result
print('train set',ann.evaluate(X_train_scaled,y_train)) # train set result

In [ ]:
ann_acc = ann.evaluate(X_test_scaled,y_test)
train_accuracy = ann.evaluate(X_train_scaled,y_train)
results.loc['ANN Model'] = [ann_acc,train_accuracy]

In [ ]:
#  Let's plot the training set and validation set losses on a graph.
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss','val_loss'])

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy','val_accuracy'],loc='best')


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# All score DataFrame
results = pd.DataFrame([],columns=['Accuracy Score','Train Accuracy'])

In [ ]:
# # Create a pipeline
# pipe = Pipeline([('scaler', StandardScaler()),('Classifier',RandomForestClassifier())])

# # Create space of candidate learning algorithms and their hyperparameters
# search_space = [{'Classifier': [LogisticRegression(max_iter=1000,solver='liblinear')],
#                  'Classifier__C': np.logspace(-2, 2, 20),
#                  'Classifier__penalty':['l1','l2']},
#                 {'Classifier': [KNeighborsClassifier()],
#                  'Classifier__n_neighbors':np.arange(1,10),
#                 'Classifier__p': [1,2]},
#                 {'Classifier': [RandomForestClassifier(random_state=20)],
#                  'Classifier__n_estimators': [300,400,500],
#                  'Classifier__max_features': ['sqrt',np.arange(6,24,3)],
#                 'Classifier__ccp_alpha':np.logspace(-3,-1,15)},
#                 {'Classifier': [GradientBoostingClassifier(random_state=10)],
#                  'Classifier__n_estimators': [100, 500, 1000],
#                  'Classifier__learning_rate':[0.001,0.01,0.1],
#                 'Classifier__ccp_alpha':np.logspace(-3,0,10)}]

# # Create grid search
# clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=3,shuffle=True,random_state=3),
#                 verbose=1,scoring="accuracy",n_jobs=-1)
# # Model fit
# best_model = clf.fit(X_train, y_train)

# # View best model
# best_model.best_estimator_.get_params()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, confusion_matrix,ConfusionMatrixDisplay

In [ ]:
# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Create model
model = GradientBoostingClassifier(ccp_alpha=0.001, learning_rate=0.01,
                            n_estimators=500, random_state=10)
# Model fit
model.fit(X_train_scaled,y_train)
# model predict
y_pred = model.predict(X_test_scaled)
acc= accuracy_score(y_test,y_pred)

print('GradientBoostingClassifier accuracy is:',acc)


In [ ]:
# train set predict
y_train_pred = model.predict(X_train_scaled)
train_acc = accuracy_score(y_train,y_train_pred)
print('train set accuracy is:',train_acc)


results.loc['GradientBoostingClassifier'] = [acc,train_acc]

In [ ]:
!pip install pydotplus

In [ ]:
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data
from IPython.display import Image

sub_tree_42 = model.estimators_[42,0]

dot_data = export_graphviz(
    sub_tree_42,
    rotate = True,
    out_file=None, filled=True, rounded=True,
    special_characters=True,
    proportion=True, impurity=True, # enable them if you want
)
graph = graph_from_dot_data(dot_data)
png = graph.create_png()
# Display
Image(png)

In [ ]:
# auc score of the multiclass model.
y_proba =np.array(model.predict_proba(X_test_scaled))
score = roc_auc_score(y_test,y_proba,multi_class='ovr',average='macro')
print('auc score of the multiclass model is:',score)

In [ ]:
# confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred),
                              display_labels=model.classes_)
disp.plot(cmap='plasma')

plt.show()

# DecisionTreeClassifier

In [ ]:
mdl = DecisionTreeClassifier()

path= mdl.cost_complexity_pruning_path(X_train,y_train)
path['ccp_alphas']

In [ ]:
# pipe = Pipeline([('scaler',StandardScaler()),('tree',DecisionTreeClassifier())])

# parameters = [{'tree':[DecisionTreeClassifier(random_state=2)],
#                'tree__max_features':np.arange(1,25),
#                'tree__ccp_alpha':path['ccp_alphas']}]

# grid = GridSearchCV(pipe,param_grid=parameters,cv=StratifiedKFold(n_splits=3,shuffle=True,random_state=0),
#                     scoring='accuracy',n_jobs=-1, verbose=2)

# best_mdl = grid.fit(X_train,y_train)

# best_mdl.best_estimator_.get_params()

In [ ]:
# standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# create model
mdl = DecisionTreeClassifier(random_state=2,max_features=18,ccp_alpha=0.002478180717510433)
# model fit
mdl.fit(X_train_scaled,y_train)
# test set predict
y_pred = mdl.predict(X_test_scaled)
# train set predict
y_tpred = mdl.predict(X_train_scaled)

train_score = accuracy_score(y_train,y_tpred)
score = accuracy_score(y_test,y_pred)
print('DecisionTreeClassifier accuracy is:',score)
print('DecisionTreeClassifier train accuracy is:',train_score)

results.loc['DecisionTreeClassifier'] = [score,train_score]

In [ ]:
from sklearn import tree

In [ ]:
# plot tree
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(mdl,filled=True)

# XGBClassifier

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# xgbc = XGBClassifier()

# pipe = Pipeline([('scaler',StandardScaler()),('boost',xgbc)])

# params = [{'boost':[XGBClassifier(objective='softmax',eval_metric='merror')],
#            'boost__max_depth': np.arange(3,10),
#            'boost__learning_rate':[0.001,0.01,0.1,0.2],
#            'boost__subsample': [0.5,1],
#            'boost__n_estimators':[100,300,500,1000]}]

# clf = GridSearchCV(pipe,params,
#                    cv=StratifiedKFold(n_splits=3,shuffle=True,random_state=1),
#                    scoring='accuracy',verbose=2,n_jobs=-1)

# best_model = clf.fit(X_train,y_train)
# best_model.best_estimator_.get_params()

In [ ]:
# standardization
scaler = StandardScaler()
X_train_reduced = scaler.fit_transform(X_train)
X_test_reduced = scaler.transform(X_test)
# create model
boost = XGBClassifier(n_estimators=1000, max_depth=9, learning_rate=0.01,
                       subsample=1,objective='multi:softprob')
# model fit
boost.fit(X_train_reduced,y_train)
# test set predict
y_pred = boost.predict(X_test_reduced)
# train set predict
train_pred = boost.predict(X_train_reduced)

xgb_train = accuracy_score(y_train,train_pred)
xgb_acc = accuracy_score(y_test,y_pred)

results.loc['XGBOOXGBClassifier'] = [xgb_acc,xgb_train]

print('XGBOOXGBClassifier accuracy is:',xgb_acc)

In [ ]:
from xgboost import plot_tree

In [ ]:
plt.figure(figsize=(20,15))
plot_tree(boost)
plt.show()

# Logistic Regression

In [ ]:
# pipe = Pipeline([('scaler',StandardScaler()),('regression',LogisticRegression())])

# mdl_params = [{'regression':[LogisticRegression(max_iter=1000,random_state=10)],
#                'regression__C': [0.001, 0.01, 0.1, 1.0, 0.0],
#                'regression__penalty':['l1','l2','elasticnet'],
#                'regression__solver': ['saga'],
#                'regression__multi_class':['multinomial']}]


# clf = GridSearchCV(pipe,mdl_params,
#                    cv=StratifiedKFold(n_splits=5,shuffle=True,random_state=1),
#                    scoring='accuracy',verbose=2,n_jobs=-1)

# best_model = clf.fit(X_train,y_train)
# best_model.best_estimator_.get_params()

In [ ]:
# standardization
scaler = StandardScaler()
X_train_reduced = scaler.fit_transform(X_train)
X_test_reduced = scaler.transform(X_test)

# Create model
logmodel = LogisticRegression(max_iter= 1000, solver = 'saga', penalty= 'l1',
                              random_state = 10, C = 1.0, multi_class = 'multinomial')
# model fit
logmodel.fit(X_train_reduced,y_train)
# test set predict
y_pred = logmodel.predict(X_test_reduced)
# train set predict
train_pred = logmodel.predict(X_train_reduced)

logmodel_train = accuracy_score(y_train,train_pred)
logmodel_acc = accuracy_score(y_test,y_pred)

results.loc['LogisticRegression'] = [logmodel_acc,logmodel_train]

print('LogisticRegression accuracy is:',logmodel_acc)


In [ ]:
results

In [ ]:
# Accuracy Score
from matplotlib import pyplot as plt
results['Accuracy Score'].plot(kind='line', figsize=(8, 4), title='Accuracy Score')
plt.gca().spines[['top', 'right']].set_visible(False)